In [1]:
import os
import numpy as np
from PIL import Image
import zipfile
import shutil
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense,Dropout, BatchNormalization
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.optimizers import Adam

In [13]:
zip_file_path = "/content/drive/MyDrive/image_dataset/shoe_boot_sandal.zip"

In [14]:
extracted_dir = "/content/temp_extracted_images"
os.makedirs(extracted_dir, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)


In [15]:


# Define the base model (pre-trained DenseNet121 without top classification layers)
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom top classification layers with increased complexity
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
num_classes=3
predictions = Dense(num_classes, activation='softmax')(x)  # Replace 'num_classes' with the number of your classes

# Create the new model
modified_model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
modified_model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [16]:
class_names = ['boot', 'sandal', 'shoe']

In [18]:
for class_name in class_names:
    output_dir = "/content/drive/MyDrive"
    class_dir = os.path.join(output_dir, class_name)
    os.makedirs(class_dir, exist_ok=True)

In [ ]:
for root, _, files in os.walk(extracted_dir):
    for file in files:
        image_path = os.path.join(root, file)
        img = image.load_img(image_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        predictions = modified_model.predict(x)
        predicted_class_index = np.argmax(predictions)
        predicted_class_name = class_names[predicted_class_index]
        destination_dir = os.path.join(output_dir, predicted_class_name)
        shutil.move(image_path, destination_dir)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 73ms/step
